In [1]:
from typing import Any, List, Mapping, Optional
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from transformers import AutoTokenizer, AutoModel, AutoConfig
import torch


def torch_gc():
    # with torch.cuda.device(DEVICE):
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    

class ChatGLM(LLM):
    max_token: int = 10000
    temperature: float = 0.01
    top_p = 0.9
    history = []
    tokenizer: object = None
    model: object = None
    history_len: int = 10

    def __init__(
        self,
        model_path: str = "chatglm-6b-int4",
        **kwargs
    ):
        super().__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_path,
            trust_remote_code=True, 
            revision=model_path
        )
        model_config = AutoConfig.from_pretrained(
            model_path, 
            trust_remote_code=True, 
            revision=model_path
        )
        self.model = AutoModel.from_pretrained(
            model_path, 
            config=model_config, 
            trust_remote_code=True, 
            revision=model_path, 
            **kwargs
        ).half().cuda()
        self.model = self.model.eval()

    @property
    def _llm_type(self) -> str:
        return "ChatGLM"

    def _call(self,
              prompt: str,
              history: List[List[str]] = [],
              stop: Optional[List[str]] = None,
              run_manager: Optional[CallbackManagerForLLMRun] = None
    ) -> str:
        response, _ = self.model.chat(
            self.tokenizer,
            prompt,
            history=history[-self.history_len:] if self.history_len > 0 else [],
            max_length=self.max_token,
            temperature=self.temperature,
        )
        torch_gc()
        history += [[prompt, response]]
        return response, history
        
    def clean_history(self,
                      **kwargs):
        self.history=[]

/root/anaconda3/envs/chat/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm = ChatGLM()

Load kernel : chatglm-6b-int4/quantization_kernels_parallel.so
Setting CPU quantization kernel threads to 2
Parallel kernel is not recommended when parallel num < 4.
Using quantization cache
Applying quantization to glm layers


In [4]:
response, history = llm._call('你好')

In [5]:
response

'你好👋！我是 ChatGLM-6B，很高兴见到你，欢迎问我任何问题。'

In [6]:
history

[['你好', '你好👋！我是人工智能助手 ChatGLM-6B，很高兴见到你，欢迎问我任何问题。'],
 ['你好', '你好👋！我是 ChatGLM-6B，很高兴见到你，欢迎问我任何问题。']]

In [7]:
response, history = llm._call('请介绍一下你自己')

In [8]:
response

'你好，我是 ChatGLM-6B，是清华大学KEG实验室和智谱AI公司于 2023 年共同训练的语言模型。我的任务是针对用户的问题和要求提供适当的答复和支持。'

In [9]:
history

[['你好', '你好👋！我是人工智能助手 ChatGLM-6B，很高兴见到你，欢迎问我任何问题。'],
 ['你好', '你好👋！我是 ChatGLM-6B，很高兴见到你，欢迎问我任何问题。'],
 ['请介绍一下你自己',
  '你好，我是 ChatGLM-6B，是清华大学KEG实验室和智谱AI公司于 2023 年共同训练的语言模型。我的任务是针对用户的问题和要求提供适当的答复和支持。']]

In [11]:
response, history = llm._call('请写一段关于PCA降维可视化的代码')

In [13]:
print(response)

好的，以下是一个简单的 Python 代码，用于展示 PCA 降维的可视化：

```python
import matplotlib.pyplot as plt
import numpy as np

# 定义数据
data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# 使用 PCA 进行降维
pca = np.power(np.power(data.reshape(-1, 1), 2), 0.5)

# 将降维后的数据和原始数据进行比较
data_pca = data[:-1] + pca
print("Original data: ", data.reshape(-1, 1))
print("Pca-降维 data: ", pca.reshape(-1, 1))

# 可视化降维
plt.imshow(pca, cmap='gray', aspect='auto', extent=[0, 10, 0, 10])
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('Pca 降维')
plt.show()
```

这个代码将原始数据(10 个变量)进行 PCA 降维，并将降维后的数据和原始数据进行比较。降维后的数据将颜色表示为该变量的值，而原始数据将颜色表示为该变量在原始数据中的分布。最后，代码将使用imshow() 函数来可视化降维。


In [17]:
response, history = llm._call('请你修改上面的代码，将原始数据中的每一个变量乘以二')

In [19]:
print(response)

好的，以下是将原始数据中的每一个变量乘以二的修改后的代码：

```python
import matplotlib.pyplot as plt
import numpy as np

# 定义数据
data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# 将原始数据中的每个变量乘以二
data_pca = data[:-1] * 2

# 将降维后的数据和原始数据进行比较
print("Original data: ", data.reshape(-1, 1))
print("Pca-降维 data: ", data_pca.reshape(-1, 1))

# 可视化降维
plt.imshow(data_pca, cmap='gray', aspect='auto', extent=[0, 10, 0, 10])
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('Pca 降维')
plt.show()
```

修改后代码将原始数据中的每个变量乘以二，并使用imshow() 函数来可视化降维。
